<a href="https://colab.research.google.com/github/arnavjune2/DL-notes/blob/main/TransferLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("fivethirtyeight")
%load_ext tensorboard

In [2]:
(X_train_full, y_train_full), (X_test, y_test)= tf.keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full/255.0
X_test = X_test/255.0
X_valid , X_train = X_train_full[:5000], X_train_full[5000:]
y_valid , y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [3]:
LAYERS = [ tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(300, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(100, kernel_initializer="he_normal"),
    tf.keras.layers.LeakyReLU(),
    tf.keras.layers.Dense(10, activation="softmax")]


model = tf.keras.models.Sequential(LAYERS)

In [4]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [6]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid), verbose=2)

Epoch 1/10
1719/1719 - 11s - loss: 1.2770 - accuracy: 0.6113 - val_loss: 0.8830 - val_accuracy: 0.7120 - 11s/epoch - 7ms/step
Epoch 2/10
1719/1719 - 5s - loss: 0.8016 - accuracy: 0.7305 - val_loss: 0.7202 - val_accuracy: 0.7624 - 5s/epoch - 3ms/step
Epoch 3/10
1719/1719 - 5s - loss: 0.6908 - accuracy: 0.7696 - val_loss: 0.6438 - val_accuracy: 0.7856 - 5s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.6307 - accuracy: 0.7884 - val_loss: 0.5983 - val_accuracy: 0.8004 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.5909 - accuracy: 0.8018 - val_loss: 0.5653 - val_accuracy: 0.8090 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 5s - loss: 0.5623 - accuracy: 0.8101 - val_loss: 0.5420 - val_accuracy: 0.8196 - 5s/epoch - 3ms/step
Epoch 7/10
1719/1719 - 5s - loss: 0.5403 - accuracy: 0.8170 - val_loss: 0.5249 - val_accuracy: 0.8252 - 5s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 5s - loss: 0.5231 - accuracy: 0.8223 - val_loss: 0.5075 - val_accuracy: 0.8358 - 5s/epoch - 3ms/step
Epoch 

In [8]:
model.save("pretrained_mnist_model.h5")

# Transfer learning 

## New Problem statement - 

Classify handwritten digits into odd and even


In [9]:
pretrained_mnist_model = tf.keras.models.load_model("pretrained_mnist_model.h5")

In [11]:
pretrained_mnist_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 10)                1010      
                                                                 
Total params: 266,610
Trainable params: 266,610
Non-trai

In [21]:
for layer in pretrained_mnist_model.layers:
  print( layer)
  print(layer.name, layer.trainable)

flatten True
dense True
leaky_re_lu True
dense_1 True
leaky_re_lu_1 True
dense_2 True


In [22]:
for layer in pretrained_mnist_model.layers[:-1]: #freezing all the layers except the last one==> trainable
  layer.trainable = False                        #freezing the layers from getting trained
  print(layer.name, layer.trainable)

flatten False
dense False
leaky_re_lu False
dense_1 False
leaky_re_lu_1 False


In [23]:
for layer in pretrained_mnist_model.layers:
  print(f"{layer.name}: {layer.trainable}")

flatten: False
dense: False
leaky_re_lu: False
dense_1: False
leaky_re_lu_1: False
dense_2: True


In [24]:
try:
  del new_model
except: pass

In [25]:
lower_pretrained_layers = pretrained_mnist_model.layers[:-1] #all layers except the last one

new_model = tf.keras.models.Sequential(lower_pretrained_layers)

new_model.add(
    tf.keras.layers.Dense(2, activation="softmax")
)

In [26]:
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                 202       
                                                                 
Total params: 265,802
Trainable params: 202
Non-traina

In [29]:
#now upadating the label for the data as we want to seperate odd one and even one!
def update_labels(labels:list):
  for index , label in enumerate(labels):
    labels[index] = np.where(label%2==0, 1 , 0)
  return labels


In [30]:
y_train_bin, y_test_bin, y_valid_bin = update_labels([y_train, y_test, y_valid])

In [33]:
np.unique(y_train_bin)

array([0, 1])

In [35]:
new_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="SGD",
    metrics=["accuracy"]
)

In [36]:
history = new_model.fit(X_train, y_train_bin, epochs=10,
                    validation_data=(X_valid, y_valid_bin), verbose=2)

Epoch 1/10
1719/1719 - 5s - loss: 0.1450 - accuracy: 0.9495 - val_loss: 0.1279 - val_accuracy: 0.9568 - 5s/epoch - 3ms/step
Epoch 2/10
1719/1719 - 4s - loss: 0.1228 - accuracy: 0.9567 - val_loss: 0.1224 - val_accuracy: 0.9610 - 4s/epoch - 2ms/step
Epoch 3/10
1719/1719 - 4s - loss: 0.1194 - accuracy: 0.9581 - val_loss: 0.1198 - val_accuracy: 0.9604 - 4s/epoch - 3ms/step
Epoch 4/10
1719/1719 - 5s - loss: 0.1175 - accuracy: 0.9588 - val_loss: 0.1191 - val_accuracy: 0.9598 - 5s/epoch - 3ms/step
Epoch 5/10
1719/1719 - 5s - loss: 0.1158 - accuracy: 0.9594 - val_loss: 0.1197 - val_accuracy: 0.9586 - 5s/epoch - 3ms/step
Epoch 6/10
1719/1719 - 4s - loss: 0.1147 - accuracy: 0.9594 - val_loss: 0.1157 - val_accuracy: 0.9616 - 4s/epoch - 2ms/step
Epoch 7/10
1719/1719 - 4s - loss: 0.1136 - accuracy: 0.9599 - val_loss: 0.1148 - val_accuracy: 0.9614 - 4s/epoch - 3ms/step
Epoch 8/10
1719/1719 - 4s - loss: 0.1129 - accuracy: 0.9601 - val_loss: 0.1139 - val_accuracy: 0.9610 - 4s/epoch - 2ms/step
Epoch 9/

In [37]:
new_model.evaluate(X_test, y_test_bin)

313/313 [==============================] - 1s 4ms/step - loss: 0.1201 - accuracy: 0.9586


[0.12007737159729004, 0.9585999846458435]